# 📊 Métricas de Evaluación para Modelos VARIMA/VAR

Una vez que un modelo VARIMA (o su versión más común, el VAR) ha sido estimado y utilizado para generar pronósticos, es crucial evaluar su rendimiento. A diferencia de la regresión lineal simple, los modelos de series de tiempo requieren métricas que penalicen los errores a lo largo de un horizonte temporal.

## 1. 📝 Fundamentos de la Evaluación de Pronósticos

En el contexto multivariado del VARIMA (donde se pronostican $K$ series simultáneamente), las métricas se aplican generalmente a los errores de pronóstico de **cada serie** individualmente.

$$
\text{Error de Pronóstico} (e_{i, t}) = \text{Valor Real} (y_{i, t}) - \text{Valor Pronosticado} (\hat{y}_{i, t})
$$

Donde $i$ es la serie (e.g., $Y$ o $X$) y $t$ es el punto en el tiempo.

## 2. 🎯 Métricas Basadas en el Error Absoluto

Estas métricas son las más fáciles de interpretar, ya que están en las mismas unidades que la serie original.

### A. Error Absoluto Medio (MAE - Mean Absolute Error)

El MAE es la media de la magnitud de los errores en el conjunto de prueba.

* **Fórmula:** Mide la desviación promedio.
    $$
    \text{MAE} = \frac{1}{N} \sum_{t=1}^{N} |e_{t}| = \frac{1}{N} \sum_{t=1}^{N} |y_{t} - \hat{y}_{t}|
    $$
* **Ventaja:** No penaliza los errores grandes tan severamente como el RMSE.
* **Interpretación:** Un MAE de 5 significa que, en promedio, el pronóstico está desviado en 5 unidades de la serie original.

### B. Error Cuadrático Medio de la Raíz (RMSE - Root Mean Squared Error)

El RMSE es la métrica más utilizada. Mide la desviación estándar de los residuos (errores de predicción).

* **Fórmula:** Se calcula la raíz cuadrada de la media de los errores al cuadrado.
    $$
    \text{RMSE} = \sqrt{\frac{1}{N} \sum_{t=1}^{N} e_{t}^2} = \sqrt{\frac{1}{N} \sum_{t=1}^{N} (y_{t} - \hat{y}_{t})^2}
    $$
* **Ventaja:** Penaliza fuertemente los errores grandes (outliers), lo que lo hace útil cuando los grandes errores son particularmente indeseables (e.g., en la gestión de riesgos).
* **Desventaja:** Al estar en las unidades de la serie, es difícil comparar modelos entre diferentes series con distintas escalas.

## 3. 🎯 Métricas Basadas en el Error Porcentual

Estas métricas son útiles para comparar la precisión de los pronósticos en diferentes series con diferentes escalas.

### A. Error Porcentual Absoluto Medio (MAPE - Mean Absolute Percentage Error)

El MAPE expresa la precisión como un porcentaje. Es intuitivo para los no expertos.

* **Fórmula:**
    $$
    \text{MAPE} = \frac{100\%}{N} \sum_{t=1}^{N} \left| \frac{y_{t} - \hat{y}_{t}}{y_{t}} \right|
    $$
* **Interpretación:** Un MAPE de 10% significa que el error de pronóstico promedio es del 10%.
* **Desventaja Crítica:** Se vuelve infinito o indefinido si el valor real ($y_t$) es cero o muy cercano a cero (un problema común en series de retornos financieros o cambios de variables).

### B. Error Porcentual Absoluto Escalonado Medio (MASE - Mean Absolute Scaled Error)

El MASE es una métrica relativamente nueva y robusta que compara el error del modelo con el error de un pronóstico ingenuo (Naive Forecast) para la misma serie.

* **Fórmula:** (El denominador es el error del pronóstico ingenuo del paso anterior, $y_{t} - y_{t-1}$)
    $$
    \text{MASE} = \frac{\frac{1}{N} \sum_{t=1}^{N} |y_{t} - \hat{y}_{t}|}{\frac{1}{N-1} \sum_{t=2}^{N} |y_{t} - y_{t-1}|}
    $$
* **Ventaja Principal:** Es independiente de la escala y es robusto ante valores de cero ($y_t=0$).
* **Interpretación Clave:**
    * **MASE < 1:** El modelo VARIMA es mejor que el pronóstico ingenuo (bueno).
    * **MASE = 1:** El modelo VARIMA es tan bueno como el pronóstico ingenuo.
    * **MASE > 1:** El modelo VARIMA es peor que el pronóstico ingenuo (malo).

## 4. 🐍 Código Python (Jupyter Notebook)

Este código muestra cómo calcular estas cuatro métricas clave utilizando los pronósticos generados por un modelo VAR.

In [2]:
# %%
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.api import VAR
from IPython.display import display, HTML, Markdown # <--- IMPORTANTE: Nuevas importaciones para una salida limpia
import warnings
warnings.filterwarnings("ignore")

# --- 1. Definición de Funciones de Métrica ---

def calculate_mape(y_true, y_pred):
    """Calcula el Error Porcentual Absoluto Medio (MAPE)."""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # Evitar división por cero o valores cercanos
    nonzero_mask = y_true != 0
    if not np.any(nonzero_mask):
         return np.nan
         
    return np.mean(np.abs((y_true[nonzero_mask] - y_pred[nonzero_mask]) / y_true[nonzero_mask])) * 100

def calculate_mase(y_true, y_pred, y_train):
    """Calcula el Error Porcentual Absoluto Escalonado Medio (MASE)."""
    # Denominador: Error Absoluto Medio del Pronóstico Naive (Naive MAE)
    naive_mae = np.mean(np.abs(y_train[1:] - y_train[:-1]))
    
    if naive_mae == 0:
        return np.inf # MASE es infinito si el pronóstico ingenuo es perfecto
        
    # Numerador: MAE del modelo sobre el conjunto de prueba
    model_mae = mean_absolute_error(y_true, y_pred)
    
    return model_mae / naive_mae

# --- 2. Preparación de Data de Prueba y Pronóstico ---

# Generación de series (mismos parámetros altamente no estacionarios)
np.random.seed(42)
N = 500
X, Y = np.zeros(N), np.zeros(N)
ruido_x, ruido_y = np.random.normal(0, 1, N), np.random.normal(0, 1, N)
for t in range(1, N):
    X[t] = X[t-1] + 0.5 * X[t-1] + ruido_x[t] * 0.1 
    Y[t] = Y[t-1] + 0.5 * Y[t-1] + 0.3 * X[t-1] * 0.1 + ruido_y[t] * 0.1

# Usamos primeras diferencias
df_var = pd.DataFrame({'Y': Y, 'X': X}).diff().dropna()

# División en entrenamiento (Train) y prueba (Test)
train_size = int(len(df_var) * 0.8)
data_train = df_var.iloc[:train_size]
data_test = df_var.iloc[train_size:]

# --- 3. Ajuste y Pronóstico del Modelo VAR ---

optimal_lag = 8 
model = VAR(data_train).fit(optimal_lag)

lag_order = model.k_ar
last_observations = data_train.values[-lag_order:]
forecast_steps = len(data_test)

forecast_values = model.forecast(last_observations, steps=forecast_steps)
forecast_df = pd.DataFrame(forecast_values, columns=['Y_pred', 'X_pred'], index=data_test.index)

print(f"✅ Modelo VAR({optimal_lag}) ajustado y pronóstico generado para {forecast_steps} pasos.")
print("-" * 50)

# --- 4. Cálculo y PRESENTACIÓN MEJORADA de Métricas ---

results = {}

for series_name in ['Y', 'X']:
    y_true = data_test[series_name].values
    y_pred = forecast_df[f'{series_name}_pred'].values
    y_train = data_train[series_name].values

    # Calcular Métricas
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = calculate_mape(y_true, y_pred)
    mase = calculate_mase(y_true, y_pred, y_train)

    # Almacenar los resultados como FLOATS (no strings) para formatear la tabla después
    results[series_name] = {
        'MAE': mae,
        'RMSE': rmse,
        'MAPE (%)': mape,
        'MASE (<1 es mejor)': mase
    }

# Convertir el diccionario a DataFrame
metrics_df = pd.DataFrame(results).T
metrics_df.index.name = 'Serie'

# --- PRESENTACIÓN FINAL LIMPIA USANDO DISPLAY ---

display(Markdown("## 📊 Resultados de las Métricas de Evaluación del Pronóstico (Modelo VAR)"))

# Formatear el DataFrame para que los números grandes se vean limpios
styled_table = metrics_df.style.format({
    'MAE': "{:.4f}",
    'RMSE': "{:.4f}",
    'MAPE (%)': "{:.2f}",
    'MASE (<1 es mejor)': "{:.4f}"
}).set_caption("Métricas de Pronóstico por Serie")

# Mostrar la tabla formateada (usa HTML internamente)
display(styled_table)

display(Markdown("\n### Interpretación del MASE (Conclusión del Modelo)"))
display(Markdown(
    f"**MASE de Y:** `{metrics_df.loc['Y', 'MASE (<1 es mejor)']:.4f}`. \n\n"
    f"**MASE de X:** `{metrics_df.loc['X', 'MASE (<1 es mejor)']:.4f}`. \n\n"
    "**Conclusión:** En nuestro ejemplo, ambos valores de MASE son extremadamente altos ($>> 1$). Esto significa que el modelo **VAR(8)** es significativamente **peor** que un simple pronóstico ingenuo ($Y_t = Y_{t-1}$) para estas series. Esto era esperado ya que la data sintética era altamente no estacionaria y el modelo VAR(8) no logró capturar la estructura subyacente de manera efectiva."
))

# %%

✅ Modelo VAR(8) ajustado y pronóstico generado para 100 pasos.
--------------------------------------------------


## 📊 Resultados de las Métricas de Evaluación del Pronóstico (Modelo VAR)

,MAE,RMSE,MAPE (%),MASE (<1 es mejor)
Serie,,,,
Y,10542088421146373446107698038437659489721237458113719662234324338240126976.0000,48267007678458888947065695023329965014989538414635194021106331667212009472.0000,0.00,1111610.1506
X,1640225453304852880750735729066410422382759428943817992288175830887038976.0000,7499082141528089771794712084960994254510335842615666966686948247400873984.0000,0.00,1496713.4933



### Interpretación del MASE (Conclusión del Modelo)

**MASE de Y:** `1111610.1506`. 

**MASE de X:** `1496713.4933`. 

**Conclusión:** En nuestro ejemplo, ambos valores de MASE son extremadamente altos ($>> 1$). Esto significa que el modelo **VAR(8)** es significativamente **peor** que un simple pronóstico ingenuo ($Y_t = Y_{t-1}$) para estas series. Esto era esperado ya que la data sintética era altamente no estacionaria y el modelo VAR(8) no logró capturar la estructura subyacente de manera efectiva.